<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - Popular MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [3]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Popular
import pandas as pd
import numpy as np

ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])
ratings.drop(['timestamp'],axis=1)
ratings.head(5)

,user,item,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Genres Matrix

In [4]:
m_cols = ['movie id','movie title','release date','video release date','IMDb URL',
          'unknown','Action','Adventure','Animation','Children','Comedy','Crime',
          'Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery',
          'Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 100k\\u.item', sep='|', names=m_cols,encoding='latin-1')
movies = movies.set_index('movie id')
movies = movies.drop(['video release date'],axis = 1)
genres = movies.drop(['movie title','release date','IMDb URL'],axis = 1)
genres

,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1679,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1680,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


# Number of genres per movie and Document Frequency

In [5]:
num_genres = genres.sum(axis=1)
df = genres.sum(axis = 0)
df = pd.DataFrame(df)
df.index.name = "Attribute"
df.columns = ['Document Frequency']
df

,Document Frequency
Attribute,
unknown,2
Action,251
Adventure,135
Animation,42
Children,122
Comedy,505
Crime,109
Documentary,50
Drama,725


# Normalize Genre Matrix by number of genres per movie

In [6]:
genres.fillna(0)
normalized = genres.divide(((((genres.T)**2).sum())**(1/2)),axis=0)
normalized

Attribute,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0.0,0.00000,0.00000,0.57735,0.57735,0.57735,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.57735,0.57735,0.00000,0.00000,0.00000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.577350,0.0,0.0
3,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0
4,0.0,0.57735,0.00000,0.00000,0.00000,0.57735,0.00000,0.0,0.577350,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.57735,0.0,0.577350,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.577350,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1679,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.707107,0.0,0.0
1680,0.0,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.707107,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.000000,0.0,0.0


# Unpivot Ratings matrix

In [7]:
ratings = ratings.pivot(index = 'user',columns='item',values ='rating')
count = ratings.count(axis=1)
ratings = ratings.fillna(0)
ratings

item,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
940,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
941,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Profile matrix (Ratings . Normalized)

In [8]:
profile = pd.DataFrame((ratings).values.dot(normalized.values), columns=normalized.columns, index=(ratings).index)
profile

Attribute,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
user,,,,,,,,,,,,,,,,,,,
1,4.0,146.915413,67.220282,26.015019,33.109273,234.707756,53.753667,24.0,328.270336,3.809401,3.535534,28.476459,21.872534,11.692388,109.110960,102.916993,119.910473,52.743606,14.731686
2,0.0,22.536304,7.064495,2.309401,7.576986,49.951537,19.720732,0.0,104.753197,1.732051,4.809401,3.621320,2.121320,7.621320,43.552990,8.089439,26.542877,5.959683,0.000000
3,0.0,22.288125,7.849332,0.000000,0.000000,24.752866,17.712296,5.0,47.854431,0.000000,2.732051,8.223615,2.568914,21.137464,9.686196,12.158733,32.293953,7.771982,0.000000
4,0.0,17.279674,7.357388,0.000000,0.000000,16.422285,12.137464,5.0,20.041087,0.000000,0.000000,2.828427,2.886751,13.118802,7.271602,12.892922,28.968045,4.236068,0.000000
5,4.0,105.038789,58.341967,31.078522,43.473999,184.271504,19.509586,0.0,50.075682,2.500000,3.535534,57.421899,23.662149,4.886751,23.858328,66.677337,31.879091,24.005723,2.886751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.0,45.160950,19.846450,1.788854,1.788854,52.794951,8.154336,0.0,63.421735,2.000000,0.000000,3.000000,1.788854,2.309401,33.112698,18.902895,32.729347,9.309037,0.000000
940,0.0,44.211123,19.020119,4.809401,12.256630,111.196662,19.995507,0.0,129.288974,0.000000,7.656854,9.654701,13.516143,7.064495,51.328315,33.955148,40.841576,28.574314,0.000000
941,0.0,23.553176,14.601908,7.562357,4.675606,20.562357,1.732051,0.0,17.192388,0.000000,0.000000,0.000000,1.788854,3.535534,2.236068,18.469908,17.049159,2.236068,0.000000


In [19]:
averaged_profile = profile.divide(profile.sum(axis=1),axis=0)
averaged_profile = averaged_profile.T
averaged_profile

user,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
Attribute,,,,,,,,,,,,,,,,,,,,,
unknown,0.002884,0.000000,0.000000,0.000000,0.005427,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Action,0.105939,0.070877,0.100383,0.114861,0.142502,0.046773,0.101546,0.264699,0.072861,0.058785,...,0.090310,0.166716,0.077924,0.072153,0.130664,0.152515,0.082890,0.174216,0.091231,0.165137
Adventure,0.048472,0.022218,0.035353,0.048906,0.079150,0.049301,0.063503,0.102814,0.052083,0.038321,...,0.061052,0.079229,0.034339,0.037466,0.065054,0.067024,0.035660,0.108006,0.064718,0.075904
Animation,0.018759,0.007263,0.000000,0.000000,0.042163,0.019709,0.015014,0.000000,0.000000,0.016444,...,0.013906,0.007760,0.003287,0.015913,0.020126,0.006041,0.009017,0.055936,0.022614,0.004487
Children,0.023875,0.023830,0.000000,0.000000,0.058979,0.035188,0.040597,0.007490,0.000000,0.018826,...,0.034185,0.007760,0.011338,0.023388,0.042008,0.006041,0.022980,0.034584,0.071946,0.017127
Comedy,0.169245,0.157099,0.111484,0.109161,0.249994,0.165410,0.112215,0.056104,0.135640,0.137389,...,0.181176,0.112760,0.205088,0.162582,0.174152,0.178296,0.208479,0.152094,0.125033,0.146721
Crime,0.038761,0.062022,0.079774,0.080679,0.026468,0.033562,0.035995,0.063117,0.000000,0.043460,...,0.022243,0.027612,0.046302,0.025140,0.026251,0.027538,0.037489,0.012811,0.000000,0.056773
Documentary,0.017306,0.000000,0.022519,0.033236,0.000000,0.003868,0.009394,0.000000,0.000000,0.011431,...,0.005928,0.000000,0.011386,0.015592,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Drama,0.236712,0.329451,0.215531,0.133216,0.067936,0.304117,0.212244,0.124785,0.248331,0.261562,...,0.222004,0.236053,0.309817,0.383101,0.159269,0.214184,0.242400,0.127167,0.216243,0.186148


# Compute Matrix X (Normalized/Document Frequency)

In [21]:
x = (normalized).divide((df.T).values)
print(x.max().max())
x

0.5


Attribute,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0.0,0.0000,0.000000,0.013746,0.004732,0.001143,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.0023,0.004277,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.002300,0.0,0.0
3,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.003984,0.0,0.0
4,0.0,0.0023,0.000000,0.000000,0.000000,0.001143,0.000000,0.0,0.000796,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.005297,0.0,0.000796,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.002300,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.001379,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1679,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.002863,0.0,0.002817,0.0,0.0
1680,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000975,0.0,0.0,0.0,0.0,0.0,0.002863,0.0,0.000000,0.0,0.0


In [22]:
xaveraged = x.divide(x.sum(axis=),axis=0)
xaveraged

Attribute,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.000000,0.700559,0.241176,0.058264,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0.0,0.259117,0.481766,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.259117,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.0,0.0
4,0.0,0.542524,0.000000,0.000000,0.000000,0.269651,0.000000,0.0,0.187826,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.631071,0.0,0.094878,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.274051,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
1679,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.504016,0.0,0.495984,0.0,0.0
1680,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.254115,0.0,0.0,0.0,0.0,0.0,0.745885,0.0,0.000000,0.0,0.0


In [24]:
gaveraged = genres.divide(genres.sum(axis=1),axis=0)
gaveraged

Attribute,unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie id,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.000000,0.333333,0.333333,0.333333,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0
4,0.0,0.333333,0.000000,0.000000,0.000000,0.333333,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1679,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.500000,0.0,0.0
1680,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.000000,0.0,0.0


# Compute Predictions (Genres . Profile^T)

In [18]:
prediction = pd.DataFrame(x.values.dot((profile.T).values), columns=(profile.T).columns, index=x.index)
prediction = prediction.T
prediction

movie id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user,,,,,,,,,,,,,,,,,,,,,
1,0.782633,0.901232,0.477731,0.867685,0.821957,0.452787,1.040696,0.686436,0.452787,0.845455,...,0.751691,0.452787,0.452787,0.452787,0.452787,0.452787,0.650168,0.632529,0.464768,0.452787
2,0.124711,0.143104,0.105749,0.192366,0.248930,0.144487,0.158802,0.176385,0.144487,0.161522,...,0.138264,0.144487,0.144487,0.144487,0.144487,0.144487,0.199458,0.226851,0.098914,0.144487
3,0.028299,0.159119,0.128661,0.117675,0.206210,0.066006,0.131797,0.066408,0.066006,0.124076,...,0.153766,0.066006,0.066006,0.066006,0.066006,0.066006,0.118707,0.074403,0.049016,0.066006
4,0.018775,0.137844,0.115411,0.074481,0.146882,0.027643,0.109811,0.034735,0.027643,0.061735,...,0.130287,0.027643,0.027643,0.027643,0.027643,0.027643,0.102425,0.040363,0.032519,0.027643
5,0.843626,0.564448,0.127008,0.492159,0.216544,0.069070,0.515652,0.456285,0.069070,0.287919,...,0.385719,0.069070,0.069070,0.069070,0.069070,0.069070,0.158110,0.117141,0.364894,0.069070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.093415,0.264040,0.130396,0.214744,0.168981,0.087478,0.194197,0.119330,0.087478,0.154567,...,0.219429,0.087478,0.087478,0.087478,0.087478,0.087478,0.186998,0.156651,0.104544,0.087478
940,0.251243,0.276981,0.162715,0.331781,0.302814,0.178330,0.363820,0.288089,0.178330,0.410677,...,0.239607,0.178330,0.178330,0.178330,0.178330,0.178330,0.261999,0.273040,0.220191,0.178330
941,0.149590,0.155841,0.067925,0.091376,0.062082,0.023714,0.146077,0.059326,0.023714,0.039038,...,0.114383,0.023714,0.023714,0.023714,0.023714,0.023714,0.054432,0.023169,0.040718,0.023714


# Normalize predictions

# Checking stuff

In [12]:
N2 = normalized.values.dot(normalized.T.values)
N2 = pd.DataFrame(N2)
N2

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,1.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.57735,0.000000
1,0.000000,1.000000,0.577350,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.816497,0.000000,0.000000,0.000000,0.000000,0.000000,0.408248,0.000000,0.00000,0.000000
2,0.000000,0.577350,1.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.707107,0.000000,0.000000,0.000000,0.000000,0.000000,0.707107,0.000000,0.00000,0.000000
3,0.333333,0.333333,0.000000,1.000000,0.333333,0.577350,0.408248,0.666667,0.577350,0.408248,...,0.408248,0.577350,0.577350,0.577350,0.577350,0.577350,0.000000,0.408248,0.57735,0.577350
4,0.000000,0.333333,0.577350,0.333333,1.000000,0.577350,0.408248,0.333333,0.577350,0.408248,...,0.408248,0.577350,0.577350,0.577350,0.577350,0.577350,0.408248,0.408248,0.00000,0.577350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0.000000,0.000000,0.000000,0.577350,0.577350,1.000000,0.707107,0.577350,1.000000,0.707107,...,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.707107,0.00000,1.000000
1678,0.000000,0.408248,0.707107,0.000000,0.408248,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000,0.00000,0.000000
1679,0.000000,0.000000,0.000000,0.408248,0.408248,0.707107,0.500000,0.408248,0.707107,0.500000,...,0.000000,0.707107,0.707107,0.707107,0.707107,0.707107,0.500000,1.000000,0.00000,0.707107
1680,0.577350,0.000000,0.000000,0.577350,0.000000,0.000000,0.000000,0.577350,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000


In [13]:
prediction = N2.values.dot(ratings.T.values)
prediction = pd.DataFrame(prediction)
prediction = prediction.divide((df.T).values)
prediction

ValueError: Unable to coerce to DataFrame, shape must be (1682, 943): given (1, 19)